In [1]:
# %cd ..
import numpy as np
import inputs, simuls, analysis
results = analysis.GetResults(verbose=True)

# PenEasy SPC simulation

In [2]:
pid = 'SPC'
# Initialize PenEasy Input Editor
peneasy_inps = inputs.PenEasy(verbose=True)

# Initialize Simulators (identified by pid)
peneasy_sims = simuls.PenEasy(verbose=True)
peneasy_sims.activate_pid('SPC')

penEasy SPC activated


In [3]:
SIZE = [20, 20, 20]
STEP = [0.3, 0.3, 0.3]  #cm
nhist = "1e6"

peneasy_inps.edit_seed(pid, 12345, 54321)
peneasy_inps.edit_source_nhist(pid, nhist)
peneasy_inps.edit_voxSize(pid, SIZE, STEP)

"penEasy/normal_nuc.in" random seeds modified to 12345 and 54321
"penEasy/normal_spc.in" random seeds modified to 12345 and 54321
"penEasy/modified.in" random seeds modified to 12345 and 54321
"penEasy/normal_nuc.in" number of histories modified to 1e6
"penEasy/normal_spc.in" number of histories modified to 1e6
"penEasy/phantomN.vox" updated to point source of activity 1e6
"penEasy/normal_nuc.in" coordinates of box center modified to (3, 3, 3)
"penEasy/normal_spc.in" coordinates of box center modified to (3, 3, 3)
"penEasy/phantomN.vox" voxel size modified to (0.3, 0.3, 0.3)


In [4]:
isotopes = ["C11", "F18", "Ga68"] 
MATS = [
    #material   mat_name   mat_id  density (g/cm3)
    {'lung' : ['lungICRP', 1,     0.30+00]},
    {'bone' : ['boneB100', 1,     1.45+00]}
        ]

peneasy_inps.verbose = False
for MAT in MATS:
    material = list(MAT.keys())[0]
    print(f"****SIMULATING {material}****")
    peneasy_inps.edit_mat(pid, MAT, SIZE, STEP)
    res_folder = f"RESULTS/SPC/{material.capitalize()}/PenEasy_xyz"
    
    for iso in isotopes:
        print(f"Simulating {iso}")
        peneasy_inps.edit_isotope(pid, iso)
        peneasy_sims.simulate(pid, get_times=True, time_samples=1, output_dir=res_folder, final_file=f"{iso}")

****SIMULATING lung****
Simulating C11
penEasy SPC real time: 1922.989 +- 0.000 s
Simulating F18
penEasy SPC real time: 1794.245 +- 0.000 s
Simulating Ga68
penEasy SPC real time: 2140.910 +- 0.000 s
****SIMULATING bone****
Simulating C11
penEasy SPC real time: 2195.662 +- 0.000 s
Simulating F18
penEasy SPC real time: 2069.445 +- 0.000 s
Simulating Ga68
penEasy SPC real time: 2447.092 +- 0.000 s


In [5]:
import numpy as np

for MAT in MATS:
    material = list(MAT.keys())[0]
    # if material == 'lung':
    #     continue
    res_folder = f"RESULTS/SPC/{material.capitalize()}/PenEasy_xyz"
    times_file = res_folder + "/" + pid + "_times.txt"
    times = np.loadtxt(times_file).astype("str")
    times[:,1], times[:,0] = times[:,0], isotopes
    np.savetxt(times_file, times, fmt='%s')

# vGATE 9.x simulation

In [ ]:
pid = '9'
# Initialize GATE Input Editor
gate_inps = inputs.GATE(verbose=True)

# Initialize Simulators (identified by pid)
gate_sims = simuls.GATE(verbose=True)
gate_sims.activate_pid(pid, output_format='dat')

In [ ]:
nhist = "1e6"

gate_inps.edit_seed(pid, 12345, 54321)
gate_inps.edit_source_nhist(pid, nhist)

In [ ]:
isotopes = ["C11", "F18", "Ga68"] 
MATS = {
    # SIZE     STEP (cm)   material   mat_name   mat_id  density (g/cm3)
    ((20,)*3, (0.3,)*3 ) : {'lung' : ['LungICRP', 2,     0.30+00]},
    ((20,)*3, (0.07,)*3) : {'bone' : ['BoneB100', 12,    1.45+00]}
        }

gate_inps.verbose = False
for STEPnSIZE, MAT in MATS.items():
    material = list(MAT.keys())[0]
    # if material == 'lung':
    #     continue
    print(f"****SIMULATING {material}****")

    SIZE, STEP = STEPnSIZE
    gate_inps.edit_voxSize(pid, SIZE, STEP)
    gate_inps.edit_mat(pid, MAT, SIZE, STEP)

    res_folder = f"RESULTS/SPC/{material.capitalize()}/GATE93_xyz"
    for iso in isotopes:
        print(f"--Simulating {iso}")
        gate_inps.edit_isotope(pid, iso)
        gate_sims.simulate(pid, get_times=True, time_samples=1, output_dir=res_folder, final_file=f"{iso}")


In [ ]:
import numpy as np

for STEPnSIZE, MAT in MATS.items():
    material = list(MAT.keys())[0]
    if material == 'lung':
        continue
    res_folder = f"RESULTS/SPC/{material.capitalize()}/GATE93_xyz"
    times_file = res_folder + "/" + pid + "_times.txt"
    times = np.loadtxt(times_file).astype("str")
    times[:,1], times[:,0] = times[:,0], isotopes
    np.savetxt(times_file, times, fmt='%s')

# Results analysis

In [9]:
program_folder = "PHITS_xyz"
mat = "Lung"
folder = f"RESULTS/SPC/{mat}/{program_folder}"
trim = int(1e6)

if mat == "Bone":
    rthresh = {"C11" : 4, "F18" : 2, "Ga68" : 8}
elif mat== "Lung":
    rthresh = {"C11" : 15, "F18" : 8, "Ga68" : 30}

for iso in rthresh.keys():
    if program_folder == "GATE7_xyz" and iso not in ["C11", "O15", "F18"]:
        continue
    print(f"Filtering {iso}")
    threshold = rthresh[iso] #mm
    analysis.filter_rmax(f"{folder}/{iso}.dat.original", f"{folder}/{iso}.dat", threshold=threshold, trim=trim, fmt='%.6f')

Filtering C11
Filtered 'RESULTS/SPC/Lung/PHITS_xyz/C11.dat.original' to 'RESULTS/SPC/Lung/PHITS_xyz/C11.dat': 0 points removed
Filtering F18
Filtered 'RESULTS/SPC/Lung/PHITS_xyz/F18.dat.original' to 'RESULTS/SPC/Lung/PHITS_xyz/F18.dat': 1 points removed
Filtering Ga68
Filtered 'RESULTS/SPC/Lung/PHITS_xyz/Ga68.dat.original' to 'RESULTS/SPC/Lung/PHITS_xyz/Ga68.dat': 5 points removed


In [3]:
import matplotlib.pyplot as plt
import numpy as np

histo_bins = 601
histo_step = .01 #cm
isotopes = ["C11", "F18", "Ga68"] 
mats = ["Lung", "Bone"]
programs = {"PHITS_xyz":"PHITS", 
            "PenEasy_xyz":"PenEasy SPC", 
            "GATE92_xyz":"GATE 9.2", 
            "GATE93_xyz":"GATE 9.3"}

PRanalysis = { mat: {iso: {p:object} for iso, p in zip(isotopes, programs.values())} for mat in mats}

results.active_results.clear()
for mat in mats:
    for iso in isotopes:
        for p in programs.keys():
            of = f"RESULTS/SPC/{mat}/{p}/{iso}.dat"
            results.load(f"{programs[p]} {iso} {mat}", of, [histo_bins]*3, [histo_step]*3)
            #PRanalysis[mat][iso][p] = analysis.PRAnalysis()

FileNotFoundError: RESULTS/SPC/Lung/PHITS_xyz/C11.dat not found.

In [ ]:
results.data_analysis()

In [ ]:
results.plot_aPSFx(sin=True, log_scale=True, lim=histo_bins*histo_step/2*10, legend_size=8)
plt.show()

In [ ]:
results.plot_aPSF3D(sin=True, log_scale=True, legend_size=8)
plt.show()

In [ ]:
results.plot_g3D(log_scale=False)

In [ ]:
results.plot_G3D()